In [268]:
import pandas as pd

Import all batting statistics. The data is broken up into Hall-of-Famers, Retired Non-Hall-of-Fame/Former All-Stars, and Active All-Stars. 

The last dataset includes every era in MLB history, with start and end years. For this one, actual MLB "eras" stopped being recorded after the Expansion era, which ended in 1973. Thus, I added makeshift eras, one between 1974 and the 21st century and one from 2000 until present 2020.

In [269]:
hof_df = pd.read_csv('data/hof_stats.csv') # Hall-of-Fame
non_hof_df = pd.read_csv('data/retired_non_hof.csv') # Retired (non-HOF, former AS)
active_df = pd.read_csv('data/active_allstars.csv') # Active All-Stars

eras_df = pd.read_csv('data/mlb_history_eras.csv') # All eras in MLB history

datasets = [hof_df, non_hof_df, active_df]

In [270]:
hof_df.head()

,Rk,Player,WAR/pos,WAA/pos,From,To,Age,G,PA,AB,...,SF,GDP,SB,CS,BA,OBP,SLG,OPS,Pos,Tm
0,1,Hank Aaron\aaronha01,143.1,93.0,1954,1976,20-42,3298,13941,12364,...,121.0,328.0,240,73.0,0.305,0.374,0.555,0.928,*9783DH/45,MLN-ATL-MIL
1,2,Pete Alexander\alexape01,3.0,3.0,1911,1930,24-43,703,1981,1810,...,NaN,NaN,3,5.0,0.209,0.242,0.275,0.517,*1/H7,PHI-CHC-STL
2,3,Roberto Alomar\alomaro01,67.0,32.5,1988,2004,20-36,2379,10400,9073,...,97.0,206.0,474,114.0,0.300,0.371,0.443,0.814,*4/HD6,SDP-TOR-BAL-CLE-NYM-CHW-ARI
3,4,Cap Anson\ansonca01,94.4,55.4,1871,1897,19-45,2524,11331,10281,...,NaN,6.0,277,16.0,0.334,0.394,0.447,0.841,*352/794681,ROK-ATH-CHC
4,5,Luis Aparicio\aparilu01,55.9,20.5,1956,1973,22-39,2599,11231,10230,...,76.0,184.0,506,136.0,0.262,0.311,0.343,0.653,*6/H,CHW-BAL-BOS


In [271]:
# total number of players 
print(len(hof_df) + len(non_hof_df) + len(active_df))

1103


This is a list of desriptions for each column name:

# Column Names and descriptions for batting stats
* **Player** - player first and last name
* **WAR/pos** - Wins Above Replacement for position players. A singular number that represents the number of wins a player adds to a team over a replacement.
* **WAA/pos** - Wins Above Average for position players. A singular number that represents the number of wins a player adds to a team over a league-average player.
* **From** -  year player's career began
* **To** -  year player's career ended
* **Age** -  age of career beginning/end on June 30 of the respective year
* **G** -  number of games player
* **PA** -  plate appearances
* **AB** -  at-bats
* **R** - runs scored
* **H** - hits
* **2B** - doubles
* **3B** - triples
* **HR** - home-runs
* **RBI** - runs batted in
* **BB** - walks
* **IBB** - intentional walks
* **SO** - strikeouts
* **HBP** - times hit by pitch
* **SH** - sacrifice hits (sacrifice bunts)
* **SF** - sacrifice flies
* **GDP** - double plays grounded into
* **SB** - stolen bases
* **CS** - number of times caught stealing
* **BA** - batting average
* **OBP** - on-base percentage
* **SLG** - slugging percentage
* **OPS** - on-base + slugging percentages
* **HOF** - indicates whether a player is in the hall-of-fame

The descriptions can also be found in 'data/batting_col_descriptions.md'

The first thing to do is add a column in each batting stats dataset that indicates whether a player is in the HOF.

In [272]:
# all players in hof_df are, by definition, in the HOF. So mark each player with a 1.
hof_df['HOF'] = 1

In [273]:
# all players in the other two datasets are not in the HOF. So mark each player with a 0.
non_hof_df['HOF'] = 0
active_df['HOF'] = 0

Next, I want to delete 'Rk' column as it is just another index. Then, I want to reorder the columns in a way that makes sense and make sure that each dataset has columns in the same order.

In [274]:
neworder = ['Player', 'From', 'To', 'Age', 'G', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 
            'BA', 'OBP', 'SLG', 'OPS', 'SB', 'CS', 'BB', 'IBB', 'SO', 'WAR/pos', 'WAA/pos', 
            'HBP', 'SH', 'SF', 'GDP', 'Pos', 'Tm', 'HOF']

hof_df = hof_df.reindex(columns=neworder)
non_hof_df = non_hof_df.reindex(columns=neworder)
active_df = active_df.reindex(columns=neworder)

The next thing to do is to drop any players whose main position was Pitcher. All of these datasets include players who have played at least 1 game at a non-Pitcher position. However, some of these players have played most of their careers at pitcher and may skew the batting statistics (traditionally, pitchers are known to be below-average hitters).

Position is indicated on a 1-9 scale:
1 (pitcher), 2 (catcher), 3 (first baseman), 4 (second baseman), 5 (third baseman), 6 (shortstop), 7 (left fielder) 8 (center fielder), and 9 (right fielder). In the datasets, the position with an * in front of it is considered a player's main position (i.e. where he played the most games). So any player with a 1 following the asterisk is a pitcher.

In [275]:
# player at index 1 is a pitcher
hof_df['Pos']

0       *9783DH/45
1            *1/H7
2           *4/HD6
3      *352/794681
4             *6/H
          ...     
185      *97D8H/35
186      *73D8/H59
187          *1/3H
188        *9/4H87
189       *68D/7H3
Name: Pos, Length: 190, dtype: object

Another thing we want to do is edit the name column, as currently they show more than just the first and last names.

In [276]:
# find all pitchers and drop them
index_names = hof_df[hof_df['Pos'].str.contains("\*1")].index
hof_df.drop(index_names, inplace = True)

index_names = non_hof_df[non_hof_df['Pos'].str.contains("\*1")].index
non_hof_df.drop(index_names, inplace = True) 

index_names = active_df[active_df['Pos'].str.contains("\*1")].index
active_df.drop(index_names, inplace = True) 

In [277]:
hof_df['Player']

0            Hank Aaron\aaronha01
2        Roberto Alomar\alomaro01
3             Cap Anson\ansonca01
4         Luis Aparicio\aparilu01
5          Luke Appling\applilu01
                  ...            
184         Hack Wilson\wilsoha01
185       Dave Winfield\winfida01
186    Carl Yastrzemski\yastrca01
188         Ross Youngs\youngro01
189         Robin Yount\yountro01
Name: Player, Length: 162, dtype: object

In [278]:
hof_df['Player'] = [name[:-10] for name in hof_df['Player']]
non_hof_df['Player'] = [name[:-10] for name in non_hof_df['Player']]
active_df['Player'] = [name[:-10] for name in active_df['Player']]

In [279]:
hof_df.head()

,Player,From,To,Age,G,PA,AB,R,H,2B,...,SO,WAR/pos,WAA/pos,HBP,SH,SF,GDP,Pos,Tm,HOF
0,Hank Aaron,1954,1976,20-42,3298,13941,12364,2174,3771,624,...,1383,143.1,93.0,32,21.0,121.0,328.0,*9783DH/45,MLN-ATL-MIL,1
2,Roberto Alomar,1988,2004,20-36,2379,10400,9073,1508,2724,504,...,1140,67.0,32.5,50,148.0,97.0,206.0,*4/HD6,SDP-TOR-BAL-CLE-NYM-CHW-ARI,1
3,Cap Anson,1871,1897,19-45,2524,11331,10281,1999,3435,582,...,330,94.4,55.4,32,34.0,NaN,6.0,*352/794681,ROK-ATH-CHC,1
4,Luis Aparicio,1956,1973,22-39,2599,11231,10230,1335,2677,394,...,742,55.9,20.5,27,161.0,76.0,184.0,*6/H,CHW-BAL-BOS,1
5,Luke Appling,1930,1950,23-43,2422,10254,8856,1319,2749,440,...,528,77.1,44.1,11,74.0,NaN,129.0,*6/5H43,CHW,1


Check for missing data in all three. Each dataframe has the exact same columns, so whatever changes are made to one, will be made to the others as well.

In [280]:
hof_df.isna().sum() / len(hof_df)

Player     0.000000
From       0.000000
To         0.000000
Age        0.000000
G          0.000000
PA         0.000000
AB         0.000000
R          0.000000
H          0.000000
2B         0.000000
3B         0.000000
HR         0.000000
RBI        0.000000
BA         0.000000
OBP        0.000000
SLG        0.000000
OPS        0.000000
SB         0.000000
CS         0.111111
BB         0.000000
IBB        0.401235
SO         0.000000
WAR/pos    0.000000
WAA/pos    0.000000
HBP        0.000000
SH         0.012346
SF         0.543210
GDP        0.290123
Pos        0.000000
Tm         0.000000
HOF        0.000000
dtype: float64

Seems IBB, SF, GDP, CS all have missing data. Here, the only one that might be useful to keep is IBB because the number of times a player has been intentionally walked could be an indication of how "feared" a hitter was. The reason that IBB is missing for so many players is that IBB was a new stat that was not measured until 1955. For now, we will drop the column but may return to include it.

In [281]:
non_hof_df.isna().sum() / len(non_hof_df)

Player     0.000000
From       0.000000
To         0.000000
Age        0.000000
G          0.000000
PA         0.000000
AB         0.000000
R          0.000000
H          0.000000
2B         0.000000
3B         0.000000
HR         0.000000
RBI        0.000000
BA         0.000000
OBP        0.000000
SLG        0.000000
OPS        0.000000
SB         0.000000
CS         0.005666
BB         0.000000
IBB        0.009915
SO         0.000000
WAR/pos    0.000000
WAA/pos    0.000000
HBP        0.000000
SH         0.000000
SF         0.143059
GDP        0.000000
Pos        0.000000
Tm         0.000000
HOF        0.000000
dtype: float64

Retired Non-Hall-of-Famers have missing values in IBB, SF, CS. All are droppable.

In [282]:
active_df.isna().sum() / len(active_df)

Player     0.0
From       0.0
To         0.0
Age        0.0
G          0.0
PA         0.0
AB         0.0
R          0.0
H          0.0
2B         0.0
3B         0.0
HR         0.0
RBI        0.0
BA         0.0
OBP        0.0
SLG        0.0
OPS        0.0
SB         0.0
CS         0.0
BB         0.0
IBB        0.0
SO         0.0
WAR/pos    0.0
WAA/pos    0.0
HBP        0.0
SH         0.0
SF         0.0
GDP        0.0
Pos        0.0
Tm         0.0
HOF        0.0
dtype: float64

No missing values for Active players.

Drop all columns with missing values. For consistency, any column dropped must be dropped in the other columns as well.

In [283]:
# dropping NA values
hof_df.drop(['IBB', 'SF', 'GDP', 'CS'], axis=1, inplace=True)
non_hof_df.drop(['IBB', 'SF', 'GDP', 'CS'], axis=1, inplace=True)
active_df.drop(['IBB', 'SF', 'GDP', 'CS'], axis=1, inplace=True)

Now we can combine the three dataframes into one large dataframe.

In [284]:
all_df = pd.concat([hof_df, non_hof_df, active_df])
all_df

,Player,From,To,Age,G,PA,AB,R,H,2B,...,SB,BB,SO,WAR/pos,WAA/pos,HBP,SH,Pos,Tm,HOF
0,Hank Aaron,1954,1976,20-42,3298,13941,12364,2174,3771,624,...,240,1402,1383,143.1,93.0,32,21.0,*9783DH/45,MLN-ATL-MIL,1
2,Roberto Alomar,1988,2004,20-36,2379,10400,9073,1508,2724,504,...,474,1032,1140,67.0,32.5,50,148.0,*4/HD6,SDP-TOR-BAL-CLE-NYM-CHW-ARI,1
3,Cap Anson,1871,1897,19-45,2524,11331,10281,1999,3435,582,...,277,984,330,94.4,55.4,32,34.0,*352/794681,ROK-ATH-CHC,1
4,Luis Aparicio,1956,1973,22-39,2599,11231,10230,1335,2677,394,...,506,736,742,55.9,20.5,27,161.0,*6/H,CHW-BAL-BOS,1
5,Luke Appling,1930,1950,23-43,2422,10254,8856,1319,2749,440,...,179,1302,528,77.1,44.1,11,74.0,*6/5H43,CHW,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,José Iglesias,2011,2020,21-30,841,3065,2848,325,792,163,...,52,135,371,12.4,2.0,44,24.0,*6/5HD4,BOS-DET-CIN-BAL,0
130,Tommy La Stella,2014,2020,25-31,531,1496,1316,164,360,72,...,5,143,159,4.4,-0.5,13,4.0,4H5/D31,ATL-CHC-LAA-OAK,0
131,Jeff McNeil,2018,2020,26-28,248,1024,918,137,293,63,...,12,69,123,9.4,6.2,29,4.0,4/795HD,NYM,0
132,Brock Hol,2012,2020,24-32,675,2401,2132,295,572,114,...,36,210,459,7.9,-0.2,23,15.0,457/H9638D1,PIT-BOS-WSN-MIL,0


Instead of dropping any column that might not be of immediate use for our model, just create a list of all column names that will be used as predictors for the model.

In [285]:
# choose statistics that will be used as predictors
predictors = ['G', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'SB',
            'BA', 'OBP', 'SLG', 'OPS', 'WAR/pos', 'WAA/pos']

# target column is HOF indicator
target_col = ['HOF']

In [289]:
all_df[predictors]

,G,AB,R,H,2B,3B,HR,RBI,BB,SB,BA,OBP,SLG,OPS,WAR/pos,WAA/pos
0,3298,12364,2174,3771,624,98,755,2297,1402,240,0.305,0.374,0.555,0.928,143.1,93.0
2,2379,9073,1508,2724,504,80,210,1134,1032,474,0.300,0.371,0.443,0.814,67.0,32.5
3,2524,10281,1999,3435,582,142,97,2075,984,277,0.334,0.394,0.447,0.841,94.4,55.4
4,2599,10230,1335,2677,394,92,83,791,736,506,0.262,0.311,0.343,0.653,55.9,20.5
5,2422,8856,1319,2749,440,102,45,1116,1302,179,0.310,0.399,0.398,0.798,77.1,44.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,841,2848,325,792,163,12,35,271,135,52,0.278,0.319,0.381,0.700,12.4,2.0
130,531,1316,164,360,72,4,31,163,143,5,0.274,0.349,0.405,0.754,4.4,-0.5
131,248,918,137,293,63,7,30,117,69,12,0.319,0.383,0.501,0.884,9.4,6.2
132,675,2132,295,572,114,16,23,211,210,36,0.268,0.337,0.369,0.707,7.9,-0.2
